# Working with Raster Bands

## Preparing Your Workspace

### Option 1: (recommended) Run in Google Colab
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kevinlacaille/presentations/blob/main/scipy2024/2_exif.ipynb)

### Option 2: Run local Jupyter instance
You can also choose to open this Notebook in your own local Jupyter instance.

**Prerequisites**

- Install: rasterio, exiftool
- Download data

In [ ]:
!pip install rasterio
!pip install PyExifTool
!wget https://raw.githubusercontent.com/kevinlacaille/presentations/main/scipy2024/data/presentation/8928dec4ddbffff/DJI_0876.JPG

In [ ]:
import rasterio

In [ ]:
import os
# Define both potential file paths
image_path = "/content/DJI_0876.JPG" if os.path.exists(
    "/content/DJI_0876.JPG"
) else "data/presentation/8928dec4ddbffff/DJI_0876.JPG"

In [ ]:
# import the image
with rasterio.open(image_path) as src:
    # read the entire image
    img = src.read()

In [ ]:
import exiftool
import json

In [ ]:
# Extract the metadata from the image
with exiftool.ExifTool() as et:
    metadata = json.loads(et.execute(b'-j', image_path))

In [ ]:
# Print the metadata json
print(json.dumps(metadata, indent=4))

In [ ]:
# Extract the GPS Altitude
altitude = float(metadata[0].get("XMP:RelativeAltitude"))
focal_length = metadata[0].get("EXIF:FocalLength")  # in mm
# Size of pixel = sensor width (m) / image width (px)
image_width = metadata[0].get("File:ImageWidth")
pixel_pitch = 6.17e-3 / image_width  # sensor width known to be 1/2.3” = 6.17mm

print("Altitude: ", altitude, "meters")
print("Focal Length: ", focal_length, "mm")
print("Pixel Pitch: ", pixel_pitch, "meters / px")

In [ ]:
# Calculate the GSD (Ground Sample Distance)
gsd = (altitude * pixel_pitch) / (focal_length / 1000)
print("GSD: ", gsd, "meters / px")
print("GSD: ", gsd * 100, "cm / px")


In [ ]:
import numpy as np

height_tree = 10  # meters

# GSD at the tree height
gsd = (altitude - height_tree) * pixel_pitch / focal_length

diameter_of_tree = 10  # meters
# Number of pixels the tree will cover
area_of_tree = diameter_of_tree / gsd
print("Area of tree: ", int(area_of_tree), "pixels")